In [10]:
# STEP 1: Import necessary packages
import pandas as pd
import os
from functools import lru_cache

# STEP 2: Display pandas version for debugging purposes
print(pd.__version__)

# STEP 3: Function to load and clean the CSV data
@lru_cache(maxsize=None)
def load_and_clean_csv(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")
    
    # STEP 3.1: Load the data
    df = pd.read_csv(file_path)
    
    # STEP 3.2: Define columns to keep
    columns_to_keep = [
        'number_of_reviews',
        'reviews_per_month',
        'minimum_nights',
        'calculated_host_listings_count',
        'review_scores_rating',
        'accommodates',
        'price',
        'availability_365',
        'availability_90',
        'availability_60', 
        'availability_30',
        'room_type',
        'room_class',
        'id',
        'latitude',
        'longitude', 
        'host_id',
        'neighbourhood_cleansed',
        'neighbourhood'
    ]
    
    # STEP 3.3: Filter DataFrame based on columns and replace NaNs with pd.NA
    existing_columns = [col for col in columns_to_keep if col in df.columns]
    df = df.loc[:, existing_columns].replace([None, ''], pd.NA)


    # STEP 3.4: Check for non-numeric values in 'price', 'latitude', and 'longitude'
    print("\nRows with non-numeric characters in 'price':")
    print(df[~df['price'].str.replace('[\$,]', '', regex=True).apply(lambda x: str(x).replace('.', '', 1).isdigit())])

    print("\nRows with non-numeric characters in 'latitude':")
    print(df[~df['latitude'].apply(lambda x: str(x).replace('.', '', 1).isdigit())])

    print("\nRows with non-numeric characters in 'longitude':")
    print(df[~df['longitude'].apply(lambda x: str(x).replace('.', '', 1).isdigit())])

    # STEP 3.5: Convert relevant columns to numeric (with coercion for errors)
    df['price'] = pd.to_numeric(df['price'].str.replace('[\$,]', '', regex=True), errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

    # STEP 3.6: Drop rows with NaNs in essential columns and print the count of deleted rows
    rows_before = df.shape[0]
    numeric_cols = ['price', 'latitude', 'longitude']
    df = df.dropna(subset=numeric_cols)
    print(f"Rows deleted: {rows_before - df.shape[0]}")

    # STEP 3.7: Clean 'price' column if it exists
    if 'price' in df.columns:
        df['price'] = df['price'].astype(str).str.replace('[\$,]', '', regex=True).astype(float)

    # STEP 3.8: Convert other columns to float where applicable
    float_cols = [
        'number_of_reviews', 'reviews_per_month', 'minimum_nights',
        'calculated_host_listings_count', 'review_scores_rating', 'accommodates',
        'availability_365', 'availability_90', 'availability_60', 'availability_30',
        'id', 'latitude', 'longitude', 'host_id'
    ]

    existing_float_cols = [col for col in float_cols if col in df.columns]

    if existing_float_cols:  # Check if any float columns exist
        df[existing_float_cols] = df[existing_float_cols].apply(pd.to_numeric, errors='coerce')
    else:
        print("Warning: None of the specified float columns are in the dataset.")


    return df

# STEP 4: Specify the file path for the raw data
file_path = 'Raw data/raw/2019-11-listings.csv'

# STEP 5: Load and clean the data
df = load_and_clean_csv(file_path)

# STEP 6: Display the first few rows of the cleaned data
print("First few rows of the cleaned data:")
print(df.head())

# STEP 7: Compare the types of values in 'room_class' and 'room_type' if they exist,
# and produce a table of values for each column
if 'room_class' in df.columns:
    print("\nRoom Class Values:")
    print(df['room_class'].value_counts())
if 'room_type' in df.columns:
    print("\nRoom Type Values:")
    print(df['room_type'].value_counts())

# STEP 8: Make a table of number of NaN or blank values per column
nan_summary = df.isna().sum()
print("\nNumber of NaN or blank values per column:")
print(nan_summary)

2.2.2


/tmp/ipykernel_15159/2270696833.py:16: DtypeWarning: Columns (0,2,6,7,9,10,11,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)



Rows with non-numeric characters in 'price':
      number_of_reviews reviews_per_month minimum_nights  \
0                    47              2.27              1   
1                   131              6.52              1   
2                     8              0.51              7   
3                     5               0.4             14   
4                    16              1.52              3   
...                 ...               ...            ...   
84499                                                      
84934                                                      
84935                                                      
85136                                                      
85137                                                      

      calculated_host_listings_count price availability_365        room_type  \
0                                  1    35              278     Private room   
1                                  3    19               23     Private r

In [15]:
# STEP 9: Keep the one that has Hotels, entire, private and shared rooms only
if 'room_class' in df.columns:
    df.drop(columns=['room_class'], inplace=True)
    print("Column 'room_class' has been deleted.")

# STEP 10: If 'room_class' exists, rename it to 'room_type'
if 'room_class' in df.columns:
    df.rename(columns={'room_class': 'room_type'}, inplace=True)
    print("Column 'room_class' has been renamed to 'room_type'.")

# STEP 11: If 'neighbourhood' exists, rename it to 'neighbourhood_cleansed'
if 'neighbourhood' in df.columns:
    df.rename(columns={'neighbourhood': 'neighbourhood_cleansed'}, inplace=True)
    print("Column 'neighbourhood' renamed to 'neighbourhood_cleansed'.")

Column 'neighbourhood' renamed to 'neighbourhood_cleansed'.


In [16]:
# STEP 9: Save the cleaned data to a new CSV file
output_file_path = 'cleaned_2019_data.csv'
df.to_csv(output_file_path, index=False)
print(f"Cleaned data has been saved to '{output_file_path}'.")

Cleaned data has been saved to 'cleaned_2019_data.csv'.


In [17]:
# STEP 1: Specify the new file path for the raw data
file_path_2021 = 'Raw data/raw/2021-10-listings.csv'

# STEP 2: Specify the new output file path for the cleaned data
output_file_path_2021 = 'cleaned_2021_data.csv'

# STEP 3:  Load and clean the data using the cached function
df_2021 = load_and_clean_csv(file_path_2021)

# STEP 4:  Display the first few rows of the cleaned data
print("First few rows of the cleaned 2021 data:")
print(df_2021.head())

First few rows of the cleaned 2021 data:
   number_of_reviews  reviews_per_month  minimum_nights  \
0                 21               0.17               1   
1                 89               0.80               3   
2                 43               0.35               4   
3                  0                NaN               3   
4                 93               0.77               7   

   calculated_host_listings_count  review_scores_rating  accommodates  price  \
0                               2                  4.85             2   65.0   
1                               1                  4.79             2   75.0   
2                              15                  4.69             6  307.0   
3                               2                   NaN             2  150.0   
4                               1                  4.89             4  195.0   

   availability_365  availability_90  availability_60  availability_30  \
0               359               84             

In [18]:
# STEP 5:  Compare the types of values in 'room_class' and 'room_type' if they exist,
# and produce a table of values for each column
if 'room_class' in df.columns:
    print("\nRoom Class Values:")
    print(df['room_class'].value_counts())
if 'room_type' in df.columns:
    print("\nRoom Type Values:")
    print(df['room_type'].value_counts())



Room Type Values:
room_type
Entire home/apt    29620
Private room       21430
Hotel room           659
Shared room          434
Name: count, dtype: int64


In [19]:
# STEP 9: Save the cleaned data to a new CSV file
df_2021.to_csv(output_file_path_2021, index=False)
print(f"Cleaned 2021 data has been saved to '{output_file_path_2021}'.")


Cleaned 2021 data has been saved to 'cleaned_2021_data.csv'.


In [20]:
# STEP 1: Specify the new file path for the raw data
file_path_2023 = 'Raw data/raw/2023-09-06-listings.csv'

# STEP 2: Specify the new output file path for the cleaned data
output_file_path_2023 = 'cleaned_2023_data.csv'

# STEP 3:  Load and clean the data using the cached function
df_2023 = load_and_clean_csv(file_path_2023)

# STEP 4:  Display the first few rows of the cleaned data
print("First few rows of the cleaned 2023 data:")
print(df_2023.head())

# STEP 5:  Compare the types of values in 'room_class' and 'room_type' if they exist,
# and produce a table of values for each column
if 'room_class' in df_2023.columns:
    print("\nRoom Class Values:")
    print(df['room_class'].value_counts())
if 'room_type' in df_2023.columns:
    print("\nRoom Type Values:")
    print(df['room_type'].value_counts())

/tmp/ipykernel_15159/2270696833.py:16: DtypeWarning: Columns (68) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)



Rows with non-numeric characters in 'price':
Empty DataFrame
Columns: [number_of_reviews, reviews_per_month, minimum_nights, calculated_host_listings_count, review_scores_rating, accommodates, price, availability_365, availability_90, availability_60, availability_30, room_type, id, latitude, longitude, host_id, neighbourhood_cleansed, neighbourhood]
Index: []

Rows with non-numeric characters in 'latitude':
Empty DataFrame
Columns: [number_of_reviews, reviews_per_month, minimum_nights, calculated_host_listings_count, review_scores_rating, accommodates, price, availability_365, availability_90, availability_60, availability_30, room_type, id, latitude, longitude, host_id, neighbourhood_cleansed, neighbourhood]
Index: []

Rows with non-numeric characters in 'longitude':
       number_of_reviews  reviews_per_month  minimum_nights  \
0                    216               1.45               2   
1                     38               0.27               5   
2                     41      

In [21]:
# STEP 9: Save the cleaned data to a new CSV file
df_2023.to_csv(output_file_path_2023, index=False)
print(f"Cleaned 2023 data has been saved to '{output_file_path_2023}'.")

Cleaned 2023 data has been saved to 'cleaned_2023_data.csv'.
